# Google Speech API 활용기 (Speech-to-Text)

>- **원하는 정확도가 나오지 않아 잠정 보류**
>- 사유는 한국어의 정확도 문제, 음원 파일의 음질문제가 (잡음 및 소리크기) 복합적으로 있었던 것으로 보임
>- 참조: https://blueriver97.tistory.com/36

# 1. 사전 준비

## API 사용 준비
- [Google Cloud Platform](https://cloud.google.com/)에서 계정 및 프로젝트 생성
- **"Cloud Speech to Text"** 검색하여 API 사용 설정
- **"서비스 계정"**에서 API 사용 가능하도록 권한 설정 (소유자, JSON키 만들어 내려받기)
- **JSON**키는 API를 호출하고 비용을 청구하는 데 사용되니 **보안에 주의하고 절대 공개 금지**

## package 및 분석 환경
- Cloud SDK를 활용해서 분석하는 사례들은 많으나, 로컬 환경을 활용하기로 함
- Python 3.7, Jupyter Notebook 사용
- 필수 package: `os`, `google-cloud-speech` (`pip install google-cloud-speech`)

# 2. 분석 코드

## 가. 환경 세팅

- Package 호출
- 사용자 인증 (JSON키 지정)

In [5]:
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="<JSON 파일 위치>"

# API 클라이언트 실행
client = speech.SpeechClient()

## 나. 1분 미만의 음성파일

- 1분 미만의 음성파일은 로컬환경에서 바로 API 적용 가능
- 1분 이상의 경우 함수도 다를 뿐더러, 로컬 파일 사용 불가
- 대신 Google Storage의 URI<sup>Uniform Resource Identifier</sup> 입력 필요

In [15]:
# 파일 불러오기
file_name = '<파일 위치>'

with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

- **파라미터 설정** `types.RecognitionConfig()`:
> `encoding`:**현재 FLAC 또는 WAV**만 사용 가능함에 따라 `ffmpeg` 사용하여 파일 변환 필요  
> `language_code`: 기본적으로 `en-US`가 가장 빈번하게 활용되며, 한국어는 `ko-KR` 사용  
> `sample_rate_hertz`: 음성 파일의 샘플레이트  
> 위 3개의 파라미터가 가장 중요하고, `enable_word_time_offsets` 외 기타 파라미터는 https://googleapis.dev/python/speech/latest/index.html 에서 reference 참조

In [18]:
config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US',
    enable_word_time_offsets=True)

- 연산 및 정확도 반환

In [19]:
response = client.recognize(config, audio)

for result in response.results:
    for alt in result.alternatives:
        print('Transcript: {}'.format(alt.transcript))
        print('Confidence: {}'.format(alt.confidence))

Transcript: OK Google stream stranger things from Netflix to my TV okay stranger things from Netflix from the people that brought you Google home comes the next evolution of a smart home and it's just outside your window me Google Now by how can I help okay he's right okay no turn on the hose I'm holding sure
Confidence: 0.9460911750793457
Transcript:  okay now I'm can I eat this lemon tree leaf yes what are the daisy yes but I wouldn't recommend it but I could eat it to my shopping list I'm sorry that sounds like an indoor request I keep doing that sorry you do keep doing that
Confidence: 0.9545512795448303
Transcript:  okay now is this compost really we're all compost if you think about it pretty much everything is made up of organic matter and will return
Confidence: 0.9341952204704285


## 다. 1분 이상의 음성파일

- 파라미터 설정은 '1분 미만의 음성파일' 분석과 동일
- 차이가 나는 부분은 1)**Google Storage**에 올라가 있는 파일을 사용해야 하며, 2)`client.recognize()` 대신 `client.long_running_recognize()`를 사용

- 파일 불러오는 방법은 `uri` 파라미터를 전달하면서 훨씬 간편하게 호출 가능

In [ ]:
audio = types.RecognitionAudio(uri = "gs://<파일 uri>")

### 이하 1분 미만의 음성파일 호출 방식
'''
file_name = "<분석용 파일 위치>"

with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)
'''

- 파라미터 설정

In [11]:
config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=8000,
    audio_channel_count = 2,
    language_code='ko-KR',
    enable_word_time_offsets=True)

- STT 구현에 `client.long_running_recognize()`를 사용하고, 실제 연산을 시키는 `.result()` 함수까지 추가됨

In [12]:
recognize = client.long_running_recognize(config, audio)
response = recognize.result(timeout = 600)

- 최종 결과 및 정확도를 확인 하는 방법은 동일

In [13]:
for result in response.results:
    for alt in result.alternatives:
        print('Transcript: {}'.format(alt.transcript))
        print('Confidence: {}'.format(alt.confidence))

Transcript: 안녕하세요 이케아 총무입니다 네네 물건을 했거든네 언니 이거 받았는데 혹시 손님 환불 하시는 이유를 좀 여쭤봐도 될까요 아 물건을 조금 주문해 가지고 아 그러세요 혹시 주문서를 제가 좀 열어 볼 수 있을까요 주문 전화번호 좀 알려주세요 잠시만요
Confidence: 0.911100447177887
Transcript: 1 6 1 9 3 1 4 7 2
Confidence: 0.6827082633972168
Transcript:  혹시 홍시만 힘드시나요 잠시만요
Confidence: 0.7154927849769592
Transcript:  지금 행 거 하나하고 옷걸이 하나를 구매를 하셨네요 그 기억이 안 받으시는 건가요 아니시면 뭐 하시는 건가요 안 맞는 것도 있고 이거를 아예 이런 거 말고
Confidence: 0.8112572431564331
Transcript:  기둥으로 돼 있어 갖고 그런 걸로 했어야 되는데 아네
Confidence: 0.7575836777687073
Transcript:  내가 자꾸 혹시 그러면 손님 그 저희 매장으로 방문해 주실 수 있으세요
Confidence: 0.82477867603302
Transcript:  지역이 대전역 가지고 아 그러시네요 죄송합니다 열심히 해요 그러시면 요거는 손님께서 부니까 10월 15일 저희 같은 경우는 교환 환불 가능 하세요 가능하시고요 그런데 지금 혹시 박스를 개봉하였거나 뭐 설치를 하셨습니까
Confidence: 0.8618425726890564
Transcript:  옥수수는 대공원에서 버렸거든요 언제 박스가 없으세요 그러시군요 저 제품 모두 다요 설치는 안 하시고 개봉하신 상태곡선 없다 레드 사무실에서 주문했는데 외환 사고 한번 가능한데 지금 포장상태는 저희가 아니면 결제로 이렇게 취소해서 환불 가능하지만 박수림 이게 그 맛이 없는 상태 면은 저희가 우선은 그 저희 그 쪽으로 그 물품이 들어와서 육안으로 어느 정도의 판별이 이후에 손님께 요것을 좋아하는 걸로 해 드릴 거예요

___
# 3. 참조사항
## 1) 음성 파일이 mp3라 진행이 안되요
- 아직까지 지원 포맷은 **wav**, 또는 **FLAC**이기 때문에 변환이 필요
- 변환은 [**ffmpeg**](http://www.ffmpeg.org/) 설치 시 cmd나 패키지를 통해 손쉽게 가능 (패키지 아님, 설치 후 path 설정 필요)
- 음성파일을 FLAC으로 변환하는 방법은 cmd에서 다음과 커맨드 실행: `ffmpeg -i sample.wav -ac 2 -c flac test.flac`
>- `-i`: 불러올 파일
>- `-ac`: 채널 갯수
>- `-c`: 코덱 설정
>- 제일 마지막에는 출력할 파일명


## 2) 음성파일이 1분을 넘어서요
- 음성파일이 1분을 초과하는데, `client.recognize()`를 사용하게 되면 `400 Inline audio exceeds duration limit. Please use a GCS URI.` 에러가 발생하게 됨
- 위에서 언급한 바와 같이 파일은 Google Storage에 업로드 한 상태로 진행 가능
- 이때 URI 주소와 `client.long_running_recognize().result()`를 사용할 필요가 있음